In [1]:
using SymPy
using Lattice
using LinearAlgebra
using SparseArrays
using ABF
using LaTeXStrings
import_from(sympy)

In [2]:
function spin_copy(H)
    H_s = zeros(eltype(H), 2size(H, 1), 2size(H, 2))
    for i in 1:size(H, 1), j in 1:size(H, 1)
        H_s[2i-1:2i, 2j-1:2j] = H[i,j]*I(2)
    end
    return H_s
end

function disorder_soc(ltc, H)
    D = spzeros(eltype(H), size(H, 1), size(H, 2))
    @assert ltc.U == 2
    m = 1
    t_am_bm =  H[index(ltc, (m, 1)), index(ltc, (m,   2))]
    t_am_amp = H[index(ltc, (m, 1)), index(ltc, (m+1, 1))]
    t_am_bmp = H[index(ltc, (m, 1)), index(ltc, (m+1, 2))]
    t_bm_amp = H[index(ltc, (m, 2)), index(ltc, (m+1, 1))]
    t_bm_bmp = H[index(ltc, (m, 2)), index(ltc, (m+1, 2))]
    
    for m in 1:ltc.N
        γ = [symbols("phi_$(m).$(i)", real = true) for i in 1:5]

        a_m = index(ltc, (m, 1))
        b_m = index(ltc, (m, 2))
        a_mp = index(ltc, (m+1, 1))
        b_mp = index(ltc, (m+1, 2))

        D[a_m, b_m]  = t_am_bm*γ[1]*im
        D[a_m, a_mp] = t_am_amp*γ[2]*im
        D[a_m, b_mp] = t_am_bmp*γ[3]*im
        D[b_m, a_mp] = t_bm_amp*γ[4]*im
        D[b_m, b_mp] = t_bm_bmp*γ[5]*im
        
        D[b_m, a_m] = D[a_m, b_m]'
        D[a_mp, a_m]= D[a_m, a_mp]'
        D[b_mp, a_m]= D[a_m, b_mp]'
        D[a_mp, b_m]= D[b_m, a_mp]'
        D[b_mp, b_m]= D[b_m, b_mp]'
    end
    return D
end

disorder_soc (generic function with 1 method)

In [3]:
@vars θ real = true
ltc = Lattice1D(20, 2)
H, U = ham_fe(ltc, Sym(-1), Sym(1), symbols("θ", real = true), pirad = false)
D = disorder_soc(ltc, H);

In [4]:
H_sf = project(U'*D*U);

H_sf_nz = SparseMatrixCSC{Bool, Int64}(iszero.(H_sf) .!= 1)
Matrix(H_sf_nz)

20×20 Matrix{Bool}:
 1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1
 1  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 1  1  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  1  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  1  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  1  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  1  1  1  1  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  1  1  1  1  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  1  1  1  1  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  1  1  1  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1  1  1  1  1  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  1  1  1  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  0  0
 0  0  0  0  0  0  0

In [5]:
#Nearest neighbor hopping
expand(H_sf[9, 10])

             4       4                   4       4                   2       6
2⋅ⅈ⋅φ_8.2⋅sin (θ)⋅cos (θ) - 2⋅ⅈ⋅φ_8.3⋅sin (θ)⋅cos (θ) + 2⋅ⅈ⋅φ_8.4⋅sin (θ)⋅cos 

                   2       6                   6       2                   2  
(θ) - 2⋅ⅈ⋅φ_8.5⋅sin (θ)⋅cos (θ) - 2⋅ⅈ⋅φ_9.1⋅sin (θ)⋅cos (θ) + 2⋅ⅈ⋅φ_9.1⋅sin (θ

     6                   4       4                   6       2                
)⋅cos (θ) + 2⋅ⅈ⋅φ_9.2⋅sin (θ)⋅cos (θ) + 2⋅ⅈ⋅φ_9.3⋅sin (θ)⋅cos (θ) - 2⋅ⅈ⋅φ_9.4⋅

   4       4                   6       2   
sin (θ)⋅cos (θ) - 2⋅ⅈ⋅φ_9.5⋅sin (θ)⋅cos (θ)

In [6]:
# Next nearest neighbor hopping
trigsimp(H_sf[9, 11])

                                       4       4   
2⋅ⅈ⋅(φ_9.2 - φ_9.3 - φ_9.4 + φ_9.5)⋅sin (θ)⋅cos (θ)